In [26]:
import numpy as np
from scipy.optimize import root_scalar, minimize, differential_evolution, NonlinearConstraint
from openpyxl import load_workbook
from consts_and_funcs import *
import random

DRONE_INITS = [FY1_INIT, FY2_INIT, FY3_INIT, FY4_INIT, FY5_INIT]
DRONE_IDS = ['FY1', 'FY2', 'FY3', 'FY4', 'FY5']
MISSILE_INITS = [M1_INIT, M2_INIT, M3_INIT]
MISSILE_IDS = ['M1', 'M2', 'M3']

# Missile velocities and distances
MISSILE_VELS = [M1_VEL, M2_VEL, M3_VEL]
MISSILE_DISTS = [M1_DIST, M2_DIST, M3_DIST]
MAX_TIME = max(M1_TIME, M2_TIME, M3_TIME)

# Shield time calculation
# drones里包含五个数组，每个数组都是一个字典包含对应参数
"""
drones = [
    {
        'theta'
        'v'
        'clouds': [
            {
                't_deploy'
                'delay'
            }
        ]
    }
]

drones_list = [drone1theta, drone1v, drone1cloud1t_deploy, ...]
"""
def dict_to_list(drones_dict):
    drones_list = []
    for drone in drones_dict:
        drones_list.append(drone['theta'])
        drones_list.append(drone['v'])
        for cloud in drone['clouds']:
            drones_list.append(cloud['t_deploy'])
            drones_list.append(cloud['delay'])
    return drones_list

def list_to_dict(drones_list):
    drones_dict = [{} for _ in range(5)]
    i = 0
    for drone in drones_dict:
        drone['theta'] = drones_list[i]
        i += 1
        drone['v'] = drones_list[i]
        i += 1
        drone['clouds'] = [{} for _ in range(3)]
        for cloud in drone['clouds']:
            cloud['t_deploy'] = drones_list[i]
            i += 1
            cloud['delay'] = drones_list[i]
            i += 1
    return drones_dict

def my_shield_time(drones):
    # 飞机方向与速度
    for i, drone in enumerate(drones):
        drone['dir_unit'] = np.array([-np.cos(drone['theta']), np.sin(drone['theta']), 0])
        drone['vel'] = drone['v'] * drone['dir_unit']
        for cloud in drone['clouds']:
            cloud['deploy_pos'] = DRONE_INITS[i] + drone['vel'] * cloud['t_deploy']
            cloud['explode_pos'] = cloud['deploy_pos'] + drone['vel'] * cloud['delay'] + np.array([0, 0, -0.5 * G * cloud['delay']**2])
            cloud['explode_t'] = cloud['t_deploy'] + cloud['delay']

    def get_cloud_center(explode_time, explode_pos):
        def cloud_center(t):
            tau = t - explode_time
            if tau < 0:
                return explode_pos + np.array([0, 0, -ESCAPE_VEL * tau])
            if tau > DURATION:
                return explode_pos + np.array([0, 0, -SINK_SPEED * DURATION]) + np.array([0, 0, -ESCAPE_VEL * (tau - DURATION)])
            else:
                return explode_pos + np.array([0, 0, -SINK_SPEED * tau])
        return cloud_center
        # Distance difference function
def get_min_distance_diff(missle_num):
    def min_distance_diff(t, cylinder_radius):
        min_distance = []
        for drone in drones:
            min_distance_1 = []
            for cloud in drone['clouds']:
                cloud_center = get_cloud_center(cloud['explode_t'], cloud['explode_pos'])
                min_distance_1.append(get_distance_diff(cloud_center, cylinder_radius, missle_num)(t))
            min_distance.append(min(min_distance_1))
        return min(min_distance)
    return min_distance_diff

def calculate_shield_time(cylinder_radius, missle_num):
    def diff(t):
        min_distance_diff = get_min_distance_diff(missle_num)
        return min_distance_diff(t, cylinder_radius)
    t_max = [M1_TIME, M2_TIME, M3_TIME][missle_num - 1]
    explode_times = []
    for drone in drones:
        for cloud in drone['clouds']:
            explode_times.append(cloud['explode_t'])
    t = min(explode_times)
    total_time = 0.0
    while t < t_max:
        while t < t_max and diff(t) > 0:
            t += 0.1
        if t >= t_max:
            break
        t_start = t
        while t < t_max and diff(t) <= 0:
            t += 0.1
        t_end = t
        if t_start != min(explode_times):
            root1 = root_scalar(diff, bracket=[t_start - 0.1, t_start], method='brentq')
            t1 = root1.root if root1.converged else t_start
        else:
            t1 = min(explode_times)
        if t_end < t_max:
            root2 = root_scalar(diff, bracket=[t_start, t_end], method='brentq')
            t2 = root2.root if root2.converged else t_end - 0.1
        else:
            t2 = t_max
        total_time += max(0, t2 - t1)
    return total_time

inner = sum([calculate_shield_time(INNER_RADIUS, missle_num) for missle_num in range(1, 4)])
outer = sum([calculate_shield_time(OUTER_RADIUS, missle_num) for missle_num in range(1, 4)])
return (inner + outer) / 2

# 计算偏心的屏蔽时间函数
def par_shield_time(theta, v, t_deploy, delay, par_id, missle_num):
    # 无人机方向（xy平面，theta是从-x轴的角度）
    dir_unit = np.array([-np.cos(theta), np.sin(theta), 0])
    drone_vel = v * dir_unit
    drone_init = [FY1_INIT, FY2_INIT, FY3_INIT, FY4_INIT, FY5_INIT][par_id - 1]
    
    # 部署位置
    deploy_pos = drone_init + drone_vel * t_deploy
    
    # 爆炸位置（烟雾弹具有无人机的水平速度，加上重力）
    explode_pos = deploy_pos + drone_vel * delay + np.array([0, 0, -0.5 * G * delay**2])
    explode_t = t_deploy + delay
    
    # 云中心函数
    def cloud_center(t):
        tau = t - explode_t
        if tau < 0:
            return explode_pos + np.array([0, 0, -ESCAPE_VEL * tau])
        if tau > DURATION:
            return explode_pos + np.array([0, 0, -SINK_SPEED * DURATION]) + np.array([0, 0, -ESCAPE_VEL * (tau - DURATION)])
        else:
            return explode_pos + np.array([0, 0, -SINK_SPEED * tau])
    
    # 内半径和外半径的平均屏蔽时间
    calculate_shield_time = get_calculate_shield_time(cloud_center, explode_t, missle_num)
    inner = calculate_shield_time(INNER_RADIUS)
    outer = calculate_shield_time(OUTER_RADIUS)
    return (inner + outer) / 2

def my_min_distance(drones):
    # 飞机方向与速度
for i, drone in enumerate(drones):
    drone['dir_unit'] = np.array([-np.cos(drone['theta']), np.sin(drone['theta']), 0])
    drone['vel'] = drone['v'] * drone['dir_unit']
    for cloud in drone['clouds']:
        cloud['deploy_pos'] = DRONE_INITS[i] + drone['vel'] * cloud['t_deploy']
        cloud['explode_pos'] = cloud['deploy_pos'] + drone['vel'] * cloud['delay'] + np.array([0, 0, -0.5 * G * cloud['delay']**2])
        cloud['explode_t'] = cloud['t_deploy'] + cloud['delay']

def get_cloud_center(explode_time, explode_pos):
    def cloud_center(t):
        tau = t - explode_time
        if tau < 0:
            return explode_pos + np.array([0, 0, -ESCAPE_VEL * tau])
        if tau > DURATION:
            return explode_pos + np.array([0, 0, -SINK_SPEED * DURATION]) + np.array([0, 0, -ESCAPE_VEL * (tau - DURATION)])
        else:
            return explode_pos + np.array([0, 0, -SINK_SPEED * tau])
    return cloud_center

# 粗略计算三个云团最小距离
def min_distance_diff_rough(t):
    min_distance = []
    for i in range(3):
        min_distance_1 = []
        for drone in drones:
            min_distance_2 = []
            for cloud in drone['clouds']:
                cloud_center = get_cloud_center(cloud['explode_t'], cloud['explode_pos'])
                min_distance_2.append(get_distance_diff_rough(cloud_center, i + 1)(t))
            min_distance_1.append(min(min_distance_2))
        min_distance.append(min(min_distance_1))
    return max(min_distance)

# 计算某个时间段里的三个云团最小距离
def min_distance(explode_t):
    t_max = max(M1_TIME, M2_TIME, M3_TIME)
    # 如果爆炸太晚，直接返回
    if t_max - explode_t <= 0.1:
        return min_distance_diff_rough(min(t_max, explode_t))
    t_arange = np.arange(explode_t, t_max, 0.1)
    distance_arange = np.array([min_distance_diff_rough(t) for t in t_arange])
    return distance_arange.min()

explode_times = []
for drone in drones:
    for cloud in drone['clouds']:
        explode_times.append(cloud['explode_t'])
return min_distance(min(explode_times))

def par_min_distance(theta, v, t_deploy, delay, par_id, missle_num):
    # 无人机方向（xy平面，theta是从-x轴的角度）
    dir_unit = np.array([-np.cos(theta), np.sin(theta), 0])
    drone_vel = v * dir_unit
    drone_init = [FY1_INIT, FY2_INIT, FY3_INIT, FY4_INIT, FY5_INIT][par_id - 1]
    
    # 部署位置
    deploy_pos = drone_init + drone_vel * t_deploy
    
    # 爆炸位置（烟雾弹具有无人机的水平速度，加上重力）
    explode_pos = deploy_pos + drone_vel * delay + np.array([0, 0, -0.5 * G * delay**2])
    explode_t = t_deploy + delay
    
    # 云中心函数
    def cloud_center(t):
        tau = t - explode_t
        if tau < 0:
            return explode_pos + np.array([0, 0, -ESCAPE_VEL * tau])
        if tau > DURATION:
            return explode_pos + np.array([0, 0, -SINK_SPEED * DURATION]) + np.array([0, 0, -ESCAPE_VEL * (tau - DURATION)])
        else:
            return explode_pos + np.array([0, 0, -SINK_SPEED * tau])
    
    return get_min_distance(cloud_center, explode_t, missle_num)

def single_bomb_shield_time(theta, v, t_deploy, delay, drone_num, missle_num):
    drone_init = DRONE_INITS[drone_num - 1]
    # 无人机方向（xy平面，theta是从-x轴的角度）
    dir_unit = np.array([-np.cos(theta), np.sin(theta), 0])
    drone_vel = v * dir_unit
    
    # 部署位置
    deploy_pos = drone_init + drone_vel * t_deploy
    
    # 爆炸位置（烟雾弹具有无人机的水平速度，加上重力）
    explode_pos = deploy_pos + drone_vel * delay + np.array([0, 0, -0.5 * G * delay**2])
    explode_t = t_deploy + delay
    # 云中心函数
def cloud_center(t):
    tau = t - explode_t
    if tau < 0:
        return explode_pos + np.array([0, 0, -ESCAPE_VEL * tau])
    if tau > DURATION:
        return explode_pos + np.array([0, 0, -SINK_SPEED * DURATION]) + np.array([0, 0, -ESCAPE_VEL * (tau - DURATION)])
    else:
        return explode_pos + np.array([0, 0, -SINK_SPEED * tau])

# 内半径和外半径的平均屏蔽时间
calculate_shield_time = get_calculate_shield_time(cloud_center, explode_t, missle_num)
inner = calculate_shield_time(INNER_RADIUS)
outer = calculate_shield_time(OUTER_RADIUS)
return (inner + outer) / 2

def objective0(params):
    drones_dict = list_to_dict(params)
    return my_min_distance(drones_dict)

def get_objective0(par_id, missle_num):
    def objective0(params):
        return par_min_distance(*params, par_id, missle_num)
    return objective0

# Objective function
def objective(params):
    drones_dict = list_to_dict(params)
    return -my_shield_time(drones_dict)

def get_objective(par_id, missle_num):
    def objective(params):
        return -par_shield_time(*params, par_id, missle_num)
    return objective

# Define constraints using NonlinearConstraint
def constraint_function(params):
    return [params[4] - params[2] - 1, params[6] - params[4] - 1,
            params[12] - params[10] - 1, params[14] - params[12] - 1,
            params[20] - params[18] - 1, params[22] - params[20] - 1,
            params[28] - params[26] - 1, params[30] - params[28] - 1,
            params[36] - params[34] - 1, params[38] - params[36] - 1]

if __name__ == '__main__':
    # Bounds for 3 drones: [theta1, v1, t_deploy1, delay1, theta2, v2, t_deploy2, delay2, theta3, v3, t_deploy3, delay3]
    bounds = [(-np.pi, np.pi), (70, 140), (0, MAX_TIME), (0, MAX_TIME), (1, MAX_TIME), (0, MAX_TIME), (2, MAX_TIME), (0, MAX_TIME),  # FY1
              (-np.pi, np.pi), (70, 140), (0, MAX_TIME), (0, MAX_TIME), (1, MAX_TIME), (0, MAX_TIME), (2, MAX_TIME), (0, MAX_TIME),  # FY2
              (-np.pi, np.pi), (70, 140), (0, MAX_TIME), (0, MAX_TIME), (1, MAX_TIME), (0, MAX_TIME), (2, MAX_TIME), (0, MAX_TIME),  # FY3
              (-np.pi, np.pi), (70, 140), (0, MAX_TIME), (0, MAX_TIME), (1, MAX_TIME), (0, MAX_TIME), (2, MAX_TIME), (0, MAX_TIME),  # FY4
              (-np.pi, np.pi), (70, 140), (0, MAX_TIME), (0, MAX_TIME), (1, MAX_TIME), (0, MAX_TIME), (2, MAX_TIME), (0, MAX_TIME)]  # FY5
    def get_bomb_bounds(par_id, missle_num):
    if par_id == 1:
        if missle_num == 1:
            return [[(-np.pi/6, np.pi/6), (70, 140), (0, 5), (0, 5)],
                    [(-np.pi/6, np.pi/6), (70, 140), (0, 5), (5, 20)],
                    [(-np.pi/6, np.pi/6), (70, 140), (5, 10), (5, 20)]]
        # N
        if missle_num == 2:
            return [[(0, np.pi/3), (70, 140), (0, 10), (0, 10)],
                    [(0, np.pi/3), (70, 140), (1, 10), (0, 20)],
                    [(0, np.pi/3), (70, 140), (2, 10), (0, 20)]]
        # N
        if missle_num == 3:
            return [[(-np.pi/2, 0), (70, 140), (0, 5), (0, 5)],
                    [(-np.pi/2, 0), (70, 140), (1, 5), (0, 20)],
                    [(-np.pi/2, 0), (70, 140), (2, 5), (0, 20)]]
    if par_id == 2:
        if missle_num == 1:
            return [[(-2*np.pi/3, 0), (70, 140), (4, 10), (5, 10)],
                    [(-2*np.pi/3, 0), (70, 140), (5, 20), (0, 20)],
                    [(-2*np.pi/3, 0), (70, 140), (6, 20), (0, 20)]]
        if missle_num == 2:
            return [[(-2*np.pi/3, 0), (70, 140), (5, 10), (0, 6)],
                    [(-2*np.pi/3, 0), (70, 140), (6, 20), (0, 20)],
                    [(-2*np.pi/3, 0), (70, 140), (7, 20), (0, 20)]]
        if missle_num == 3:
            return [[(-2*np.pi/3, 0), (70, 140), (5, 10), (5, 8)],
                    [(-2*np.pi/3, 0), (70, 140), (6, 20), (0, 20)],
                    [(-2*np.pi/3, 0), (70, 140), (7, 20), (0, 20)]]
    if par_id == 3:
        if missle_num == 1:
            return [[(np.pi/3, 2*np.pi/3), (70, 140), (20, 30), (0, 10)],
                    [(np.pi/3, 2*np.pi/3), (70, 140), (21, 35), (0, 20)],
                    [(np.pi/3, 2*np.pi/3), (70, 140), (22, 40), (0, 20)]]
        if missle_num == 2:
            return [[(np.pi/4, np.pi/2), (70, 140), (20, 30), (0, 10)],
                    [(np.pi/4, np.pi/2), (70, 140), (21, 30), (0, 20)],
                    [(np.pi/4, np.pi/2), (70, 140), (22, 35), (0, 20)]]
        if missle_num == 3:
            return [[(np.pi/4, 3*np.pi/4), (70, 140), (20, 30), (0, 10)],
                    [(np.pi/4, 3*np.pi/4), (70, 140), (21, 30), (0, 20)],
                    [(np.pi/4, 3*np.pi/4), (70, 140), (22, 35), (0, 20)]]
    if par_id == 4:
        if missle_num == 1:
            return [[(-np.pi/2, -np.pi/6), (70, 140), (0, 10), (10, 20)],
                    [(-np.pi/2, -np.pi/6), (70, 140), (1, 20), (0, 20)],
                    [(-np.pi/2, -np.pi/6), (70, 140), (2, 30), (0, 30)]]
        if missle_num == 2:
            return [[(-np.pi, -np.pi/6), (70, 140), (0, 10), (5, 15)],
                    [(-np.pi, -np.pi/6), (70, 140), (1, 20), (0, 20)],
                    [(-np.pi, -np.pi/6), (70, 140), (2, 20), (0, 30)]]
        if missle_num == 3:
            return [[(-np.pi/2, -np.pi/6), (70, 140), (5, 15), (10, 20)],
                    [(-np.pi/2, -np.pi/6), (70, 140), (6, 20), (0, 30)],
                    [(-np.pi/2, -np.pi/6), (70, 140), (7, 30), (0, 30)]]
    if par_id == 5:
        if missle_num == 1:
            return [[(np.pi/4, np.pi/2), (70, 140), (15, 25), (0, 5)],
                    [(np.pi/4, np.pi/2), (70, 140), (16, 35), (0, 15)],
                    [(np.pi/4, np.pi/2), (70, 140), (17, 40), (0, 20)]]
        if missle_num == 2:
            return [[(np.pi/6, np.pi/2), (70, 140), (15, 25), (0, 5)],
                    [(np.pi/6, np.pi/2), (70, 140), (16, 40), (0, 20)],
                    [(np.pi/6, np.pi/2), (70, 140), (17, 45), (0, 20)]]
        if missle_num == 3:
            return [[(np.pi/6, np.pi/2), (70, 140), (12, 20), (0, 5)],
                    [(np.pi/6, np.pi/2), (70, 140), (13, 30), (0, 20)],
                    [(np.pi/6, np.pi/2), (70, 140), (14, 40), (0, 20)]]

# Nonlinear constraints: each constraint >= 0
constraints = NonlinearConstraint(constraint_function, 0, np.inf)
# Nonlinear constraints: each constraint >= 0
constraints = NonlinearConstraint(constraint_function, 0, np.inf)

# Callback to monitor optimization progress
iteration = 0
def callback(xk, convergence):
    global iteration
    iteration += 1
    if iteration % 10 == 0:  # Print every 10 iterations
        current_time = -objective(xk)
        print(f"Iteration {iteration}: Best shielding time = {current_time:.3f} s.")

def get_callback(par_id, missle_num, bomb_id):
    def callback(xk, convergence):
        global iteration
        iteration += 1
        if iteration % 10 == 0:  # Save every 10 iterations
            current_time = -get_objective(par_id, missle_num)(xk)
            theta, v, t_deploy, delay = xk
            log_message = (f"Iteration {iteration}: Best shielding time = {current_time:.3f} s, par_id: {par_id}, "
                          f"missle_num: {missle_num}, bomb_id: {bomb_id}, theta = {np.degrees(theta) % 360:.3f}°, "
                          f"v = {v:.3f} m/s, t_deploy = {t_deploy:.3f} s, delay = {delay:.3f} s\n")
            print(log_message.strip())
    return callback

# 类似demo3进行局部求解
def only_for_you(par_id, missle_num, sample_num):
    only_bounds = bounds[par_id * 8 - 8: par_id * 8]
    def only_constraint_function(params):
        _, _, t_deploy1, _, t_deploy2, _, t_deploy3, _ = params
        return [t_deploy2 - t_deploy1 - 1, t_deploy3 - t_deploy2 - 1]
    only_constraints = NonlinearConstraint(only_constraint_function, 0, np.inf)
    result0 = [[] for _ in range(3)]
    for bomb_id in range(1, 4):
        for _ in range(sample_num):
            a_result0 = differential_evolution(
                get_objective0(par_id, missle_num),
                get_bomb_bounds(par_id, missle_num)[bomb_id - 1],
                strategy='best1bin',
                maxiter=10,
                popsize=5,
                tol=1e-3,
                callback=get_callback(par_id, missle_num, bomb_id),
                init='random',
                disp=False,
                polish=False
            )
            result0[bomb_id - 1].append(a_result0.x)

    # 提取20个解作为第二步的初始种群，起始的速度与方向平均分配
    initial_population = np.array([*[[*a, *b[2:], *c[2:]] for a, b, c in list(zip(result0[0], result0[1], result0[2]))[0: sample_num // 3]],
                                  *[[*b[:2], *a[2:], *b[2:], *c[2:]] for a, b, c in list(zip(result0[0], result0[1], result0[2]))[sample_num // 3: 2 * sample_num // 3]],
                                  *[[*c[:2], *a[2:], *b[2:], *c[2:]] for a, b, c in list(zip(result0[0], result0[1], result0[2]))[2 * sample_num // 3:]]])

    def only_shield_time(theta, v, t_deploy1, delay1, t_deploy2, delay2, t_deploy3, delay3):
        drone_init = DRONE_INITS[par_id - 1]
        dir_unit = np.array([-np.cos(theta), np.sin(theta), 0])
        drone_vel = v * dir_unit

        deploy_positions = []
        explode_positions = []
        explode_times = []
        for t_deploy, delay in [(t_deploy1, delay1), (t_deploy2, delay2), (t_deploy3, delay3)]:
            deploy_pos = drone_init + drone_vel * t_deploy
            explode_pos = deploy_pos + drone_vel * delay + np.array([0, 0, -0.5 * G * delay**2])
            explode_t = t_deploy + delay
            deploy_positions.append(deploy_pos)
            explode_positions.append(explode_pos)
            explode_times.append(explode_t)

        # Cloud center function for each bomb
        def get_cloud_center(i):
            def cloud_center(t):
                tau = t - explode_times[i]
                if tau < 0:
                    return explode_positions[i] + np.array([0, 0, -ESCAPE_VEL * tau])
                if tau > DURATION:
                    return explode_positions[i] + np.array([0, 0, -SINK_SPEED * DURATION]) + np.array([0, 0, -ESCAPE_VEL * (tau - DURATION)])
                else:
                    return explode_positions[i] + np.array([0, 0, -SINK_SPEED * tau])
            return cloud_center
            # Distance difference function
def min_distance_diff(t, cylinder_radius):
    min_distance = []
    for i in range(3):
        cloud_center = get_cloud_center(i)
        min_distance.append(get_distance_diff(cloud_center, cylinder_radius, missle_num)(t))
    return min(min_distance)

# Calculate total shielding time for a given radius
def calculate_shield_time(cylinder_radius):
    def diff(t):
        return min_distance_diff(t, cylinder_radius)
    
    t_max = [M1_TIME, M2_TIME, M3_TIME][missle_num - 1]
    t = min(explode_times)
    total_time = 0.0
    while t < t_max:
        while t < t_max and diff(t) > 0:
            t += 0.1
        if t >= t_max:
            break
        t_start = t
        while t < t_max and diff(t) <= 0:
            t += 0.1
        t_end = t
        if t_start != min(explode_times):
            root1 = root_scalar(diff, bracket=[t_start - 0.1, t_start], method='brentq')
            t1 = root1.root if root1.converged else t_start
        else:
            t1 = min(explode_times)
        if t_end < t_max:
            root2 = root_scalar(diff, bracket=[t_start, t_end], method='brentq')
            t2 = root2.root if root2.converged else t_end - 0.1
        else:
            t2 = t_max
        total_time += max(0, t2 - t1)
    return total_time

inner = calculate_shield_time(INNER_RADIUS)
outer = calculate_shield_time(OUTER_RADIUS)
return (inner + outer) / 2

def only_objective(params):
    return -only_shield_time(*params)

def only_callback(xk, convergence):
    global iteration
    iteration += 1
    if iteration % 10 == 0:  # Save every 10 iterations
        current_time = -only_objective(xk)
        theta, v, t_deploy1, delay1, _, _, _, _ = xk
        log_message = (f"Iteration {iteration}: Best shielding time = {current_time:.3f} s, par_id: {par_id}"
                      f"missle_num: {missle_num}, theta = {np.degrees(theta) % 360:.3f}°, v = {v:.3f} m/s, "
                      f"t_deploy1 = {t_deploy1:.3f} s, delay1 = {delay1:.3f} s\n")
        print(log_message.strip())
        # 第二步：使用第一步的结果作为初始种群再次运行differential_evolution
a_result = differential_evolution(
    only_objective,
    only_bounds,
    strategy='best1bin',  # 可根据需要调整策略
    constraints=only_constraints,
    maxiter=200,  # 可增加迭代次数以进一步优化
    popsize=sample_num,  # 设置为5，与初始种群大小匹配
    tol=1e-3,
    callback=only_callback,
    init=initial_population,  # 使用第一步的结果作为初始种群
    disp=False,
    polish=False
)

return a_result.x

result1 = [[] for _ in range(5)]
for par_id in range(1, 6):
    for missle_num in range(1, 4):
        for _ in range(5):
            result1[par_id - 1].append(only_for_you(par_id, missle_num, 6))

for res in result1:
    random.shuffle(res)

result0 = [[*a, *b, *c, *d, *e] for a, b, c, d, e in zip(result1[0], result1[1], result1[2], result1[3], result1[4])]
# 提取20个解作为第二步的初始种群
initial_population = np.array(result0)
result = differential_evolution(
    objective,
    bounds,
    strategy='best1bin',  # 可根据需要调整策略
    constraints=constraints,
    maxiter=200,  # 可增加迭代次数以进一步优化
    popsize=15,  # 设置为15，与初始种群大小匹配
    tol=1e-3,
    callback=callback,
    init=initial_population,  # 使用第一步的结果作为初始种群
    disp=False,
    polish=False
)

# Prepare data
data = list_to_dict(result.x)
opt_time = -result.fun

for i, drone in enumerate(data):
    drone_id = DRONE_IDS[i]
    theta = drone['theta']
    v = drone['v']
    print(f"\nDrone {drone_id}:")
    print(f" 运动方向 (theta): {theta:.3f} 弧度")
    print(f" 运动速度 (v): {v:.3f} m/s")
    for j, cloud in enumerate(drone['clouds']):
        t_deploy = cloud['t_deploy']
        delay = cloud['delay']
        explode_t = t_deploy + delay
        print(f" 空气弹 {j+1}:")
        print(f" 投放时间 (t_deploy): {t_deploy:.3f} s")
        print(f" 起爆时间 (explode_t): {explode_t:.3f} s")
        # 打印最优屏蔽时间
print(f"\n最优屏蔽时间: {opt_time:.3f} s")
file_path = 'A题/附件/result3.xlsx'
workbook = load_workbook(file_path)

# 选择工作表（默认第一个sheet，也可以用sheet名称）
sheet = workbook.active  # 或者 workbook["Sheet1"]

for row in range(2, 17):
    fy = (row + 1) // 3
    bomb = (row + 2) % 3
    theta_deg = np.degrees(data[fy - 1]['theta']) % 360
    opt_v = data[fy - 1]['v']
    opt_t_deploy = data[fy - 1]['clouds'][bomb - 1]['t_deploy']
    opt_delay = data[fy - 1]['clouds'][bomb - 1]['delay']
    dir_unit = np.array([-np.cos(data[fy - 1]['theta']), np.sin(data[fy - 1]['theta']), 0])
    vel = opt_v * dir_unit
    deploy_pos = DRONE_INITS[fy - 1] + vel * opt_t_deploy
    explode_pos = deploy_pos + vel * opt_delay + np.array([0, 0, -0.5 * G * opt_delay**2])
    sheet.cell(row=row, column=2).value = round(theta_deg, 3)
    sheet.cell(row=row, column=3).value = round(opt_v, 3)
    sheet.cell(row=row, column=5).value = round(deploy_pos[0], 3)
    sheet.cell(row=row, column=6).value = round(deploy_pos[1], 3)
    sheet.cell(row=row, column=7).value = round(deploy_pos[2], 3)
    sheet.cell(row=row, column=8).value = round(explode_pos[0], 3)
    sheet.cell(row=row, column=9).value = round(explode_pos[1], 3)
    sheet.cell(row=row, column=10).value = round(explode_pos[2], 3)
    alist = []
    for mis in range(1, 4):
        alist.append(single_bomb_shield_time(data[fy - 1]['theta'], data[fy - 1]['v'], opt_t_deploy, opt_delay, fy, mis))
    sheet.cell(row=row, column=11).value = round(max(alist), 3)
    for ind, dt in enumerate(alist):
        if dt > 1e-6:
            sheet.cell(row=row, column=12).value = ind + 1

# 保存文件
workbook.save(file_path)
print(f"已将内容写入 {file_path}.")

IndentationError: expected an indented block after function definition on line 196 (2097022803.py, line 197)